In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

C:\Users\Gerso\Anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
f = h5py.File('simulation SXSBBH1369.h5', 'r')
df = pd.DataFrame()

In [3]:
for x in f.values():
    dset = f[x.name].items()
    data = pd.DataFrame(dset)
    data = data.drop([38,61,64,1]).sort_values(0)
    data.reset_index(drop = True, inplace = True)
    names = data[0]
    list = data[1]
    for index, l in enumerate(list):
        arr = np.zeros((14891,3), dtype='float64')
        l.read_direct(arr)
        list[index] = pd.DataFrame(arr).drop(0,axis=1).rename(index=str, columns={1: 'coluna_1', 2: 'coluna_2'})
    data = data.rename(index=str, columns={1: x.name})
    df[x.name] = data[x.name]

In [4]:
describe = df.copy()
for i in range(0,len(describe.columns)):
    for j in range(0,len(describe[df.columns[i]])):
        describe[df.columns[i]][j] = describe[df.columns[i]][j].describe().rename(
            index={
                'std': 'Desvio Padrão',
                'mean': 'Média',
                'count': 'Quantidade',
                'min': 'Mínimo',
                'max': 'Máximo',
                '50%': 'Mediana'
            })

In [5]:
info_1 = pd.DataFrame()
info_2 = pd.DataFrame()
for a in describe[describe.columns[0]][0].index:
    info_1[a] = 0
    info_2[a] = 0

In [6]:
for i in range(0,len(describe.columns)):
    for j in range(0,len(describe[df.columns[i]])):
        d_1 = pd.DataFrame(describe[describe.columns[i]][j]['coluna_1'])
        d_1 = d_1.transpose()
        d_1.reset_index(drop = True, inplace = True)
        info_1 = info_1.append(d_1,ignore_index=True)
        
for i in range(0,len(describe.columns)):
    for j in range(0,len(describe[df.columns[i]])):
        d_2 = pd.DataFrame(describe[describe.columns[i]][j]['coluna_2'])
        d_2 = d_2.transpose()
        d_2.reset_index(drop = True, inplace = True)
        info_2 = info_2.append(d_2,ignore_index=True)

In [7]:
csv = pd.DataFrame()
csv['Mínimo_col1'] = info_1.min()
csv['Máximo_col1'] = info_1.max()
csv['Mínimo_col2'] = info_2.min()
csv['Máximo_col2'] = info_2.max()
csv

,Mínimo_col1,Máximo_col1,Mínimo_col2,Máximo_col2
Quantidade,1.489100e+04,14891.000000,1.489100e+04,14891.000000
Média,-6.552916e-04,0.003802,-8.140885e-04,0.000814
Desvio Padrão,8.213068e-08,0.098114,3.515221e-07,0.098237
Mínimo,-3.194235e-01,-0.000003,-3.409977e-01,-0.000010
25%,-7.280722e-02,0.000416,-7.347323e-02,0.000043
Mediana,-1.361203e-04,0.000508,-1.362534e-04,0.000136
75%,-8.052952e-05,0.072615,-4.345053e-05,0.073473
Máximo,1.888276e-06,0.347204,1.022112e-05,0.340998


In [8]:
csv.to_csv('estatisticas SXSBBH1369.csv')